# Space charge instabilities 

In [ ]:
#hide
import sys

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import proplot as plot
import seaborn as sns
from tqdm import tqdm

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import plotting as myplt
from tools import animation as myanim
from tools import beam_analysis as ba
from tools import utils

In [ ]:
#hide
plot.rc['figure.facecolor'] = 'white'
plot.rc['grid.alpha'] = 0.04
plot.rc['axes.grid'] = False
plot.rc['animation.html'] = 'jshtml'
plot.rc['savefig.dpi'] = 150
plot.rc['savefig.facecolor'] = 'white'

## Incoherent fourth-order space charge resonance 

In [ ]:
#hide
coords = np.load('_saved/gaussian_tune100-90/coords.npy')
tmax = 300
coords = coords[:tmax+1]
coords *= 1e3
xmax = np.max(coords[:, :, 0])
xpmax = np.max(coords[:, :, 1])
xmax, xpmax = 1.15 * np.array([xmax, xpmax])

In [ ]:
#hide
moments = 1e6 * np.load('_saved/gaussian_tune100-90/moments.npy')
stats = ba.Stats(1)
stats.read_moments(moments)

In [ ]:
#hide
eps_min = np.min(stats.twiss2D.loc[:tmax+1, ['ex','ey']].values)
eps_max = np.max(stats.twiss2D.loc[:tmax+1, ['ex','ey']].values)
pad = 5.0 # [mm mrad]

In [ ]:
#hide_input
fig, axes = plot.subplots([[1, 2], [1, 3]], figsize=(6, 3), sharex=False, sharey=False, wspace=0.75, 
                          height_ratios=[1, 0.2], width_ratios=[1, 0.75])
axes[0].format(xlim=(-xmax, xmax), ylim=(-xpmax, xpmax), xlabel='x [mm]', ylabel="x' [mrad]")
axes[1].format(xlim=(0, len(coords)), ylim=(eps_min - pad, eps_max + pad),
               xlabel='Cell', ylabel='[mm mrad]')
myplt.despine([axes[0]])
line1, = axes[0].plot([], [], lw=0, markeredgewidth=0, ms=0.2, marker='o', c='k')
line2, = axes[1].plot([], [])
line3, = axes[1].plot([], [])
axes[2].axis('off')
cells = list(range(len(coords)))
plt.close()

stride = 3

def update(i):
    i *= stride
    X = coords[i]
    line1.set_data(X[:, 0], X[:, 1])
    axes[0].set_title('Cell {}'.format(i))
    data = stats.twiss2D[['ex','ey']].values
    line2.set_data(cells[:i], data[:i, 0])
    line3.set_data(cells[:i], data[:i, 1])
    axes[1].legend(labels=[r'$\varepsilon_x$', r'$\varepsilon_y$'], ncols=1, handlelength=1.5, loc='upper left')
    
frames = int(tmax / stride)
frames = 10
anim = animation.FuncAnimation(fig, update, frames=frames, interval=1000/10)
anim

In [ ]:
anim.save('test.mov', dpi=500)

## Envelope instability

In [ ]:
#hide_input
depressed_tunes = np.load('_saved/envelope/depressed_tunes.npy')
sizes_arr = [1e3 * np.load('_saved/envelope/sizes_{}.npy'.format(i)) for i in range(2)]
fig, ax = plot.subplots(figsize=(6, 2))
lw = 0.7
ax.plot(sizes_arr[1][:, 0], lw=lw)
ax.plot(sizes_arr[1][:, 1], lw=lw)
ax.format(xlabel='Cell number', ylabel='[mm]',
          xlim=(0, len(sizes_arr[0])))
ax.legend(labels=[r'$\sqrt{\langle{x^2}\rangle}$', r'$\sqrt{\langle{y^2}\rangle}$'], 
          fontsize='small', ncols=1, loc='upper left', lw=1.5)
ax.set_title(r'Depressed phase advance per cell = {:.0f} degrees'.format(depressed_tunes[1]))
plt.savefig('_saved/env_instability.png', facecolor='white', dpi=300)

## Coherent instability

In [ ]:
#hide
coords0 = np.load('_saved/mu0-90_mu-45_kv/coords.npy')
coords1 = np.load('_saved/mu0-90_mu-30_kv/coords.npy')
coords2 = np.load('_saved/mu0-90_mu-45_wb/coords.npy')
coords3 = np.load('_saved/mu0-90_mu-30_wb/coords.npy')
coords4 = np.load('_saved/mu0-90_mu-45_gauss/coords.npy')
coords5 = np.load('_saved/mu0-90_mu-30_gauss/coords.npy')

coords_list = [coords0, coords2, coords4, coords1, coords3, coords5]

moments0 = np.load('_saved/mu0-90_mu-45_kv/moments.npy')
moments1 = np.load('_saved/mu0-90_mu-30_kv/moments.npy')
moments2 = np.load('_saved/mu0-90_mu-45_wb/moments.npy')
moments3 = np.load('_saved/mu0-90_mu-30_wb/moments.npy')
moments4 = np.load('_saved/mu0-90_mu-45_gauss/moments.npy')
moments5 = np.load('_saved/mu0-90_mu-30_gauss/moments.npy')

moments_list = [moments0, moments2, moments4, moments1, moments3, moments5]
stats_list = []
for moments in moments_list:
    stats = ba.Stats(1)
    stats.read_moments(moments)
    stats_list.append(stats)

### Emittance growth 

In [ ]:
#hide_input
fig, axes = plot.subplots(nrows=2, ncols=3, figsize=(7, 3))
myplt.despine(axes, ('top', 'right'))
axes.format(xlabel="Cell", ylabel="[mm mrad]", suptitle='Emittance growth', grid=True)
for ax, title in zip(axes[0, :], ['KV', 'Waterbag', 'Gaussian']):
    ax.set_title(title)
axes[0, -1].annotate(r'$\nu_{x} = 45\degree$', xy=(1, 0.5), xycoords='axes fraction')
axes[1, -1].annotate(r'$\nu_{x} = 30\degree$', xy=(1, 0.5), xycoords='axes fraction')

for ax, stats in zip(axes, stats_list):
    stats.twiss2D[['ex','ey']].plot(ax=ax, legend=False)
    
axes[1, -1].legend(labels=[r'$\varepsilon_x$', r'$\varepsilon_y$'],
                   ncols=2, handlelength=1, fontsize='small', loc='upper left');

### Horizontal phase space

In [ ]:
#hide
xmax = max([np.max(coords[:, :, 0]) for coords in coords_list])
xpmax = max([np.max(coords[:, :, 1]) for coords in coords_list])

In [ ]:
#hide_input
fig, axes = plot.subplots(nrows=2, ncols=3, figsize=(6, 3.5))
myplt.despine(axes, ('top', 'right'))

axes.format(xlim=(-xmax, xmax), ylim=(-xpmax, xpmax),
            xlabel="x [mm]", ylabel="x' [mrad]",
            suptitle='Horizontal phase space evolution')
for ax, title in zip(axes[0, :], ['KV', 'Waterbag', 'Gaussian']):
    ax.set_title(title)
axes[0, -1].annotate(r'$\nu_{x} = 45\degree$', xy=(1, 0.5), xycoords='axes fraction')
axes[1, -1].annotate(r'$\nu_{x} = 30\degree$', xy=(1, 0.5), xycoords='axes fraction')

kws = dict(lw=0, marker='o', markeredgecolor='none', ms=0.2, c='k')
line0, = axes[0, 0].plot([], [], **kws)
line1, = axes[0, 1].plot([], [], **kws)
line2, = axes[0, 2].plot([], [], **kws)
line3, = axes[1, 0].plot([], [], **kws)
line4, = axes[1, 1].plot([], [], **kws)
line5, = axes[1, 2].plot([], [], **kws)
lines = [line0, line1, line2, line3, line4, line5]
plt.close()

def update(i):
    for ax, line, coords in zip(axes, lines, coords_list):
        X = coords[i]
        line.set_data(X[:, 0], X[:, 1])
    myplt.remove_annotations(axes[0])
    axes[0].annotate('cell {}'.format(i), xy=(0.78, 0.91), xycoords='axes fraction')

n_frames = len(coords1)
anim = animation.FuncAnimation(fig, update, frames=n_frames, interval=1000/2)
anim